<a href="https://colab.research.google.com/github/nikhilpanda004/CourseraCapstone/blob/master/POCkind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-23 06:47:55.833792: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-23 06:47:55.834177: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-23 06:47:55.834225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [36]:
# Import required libraries
import spacy
from spacy.matcher import Matcher
import re

In [42]:
# Load the pre-trained spacy model
nlp = spacy.load("en_core_web_lg")
print(nlp('Hello World!!!'))

/usr/local/lib/python3.8/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_lg' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Hello World!!!


In [39]:
# Define the pre-loaded queries as a dictionary
pre_loaded_queries = {
    'match_all_nodes': 'MATCH (n) RETURN n',
    'match_nodes_by_label': 'MATCH (n:{label}) RETURN n',
    'match_nodes_by_property': 'MATCH (n) WHERE n.{property} = "{value}" RETURN n',
    'match_nodes_by_label_and_property': 'MATCH (n:{label}) WHERE n.{property} = "{value}" RETURN n'
}

In [40]:
# Define the function to generate the Cypher query from natural language input
def generate_cypher_query(natural_language_input):
    # Define the variable to store the generated query
    cypher_query = ''
    
    # Tokenize the input text and apply POS tagging
    doc = nlp(natural_language_input)
    print('doc: ', doc)
    # Initialize the matcher and add pattern to match the labels
    matcher = Matcher(nlp.vocab)
    pattern = [{"LOWER": "label"}, {"IS_PUNCT": True}, {"POS": "PROPN"}]
    matcher.add("Label", [pattern])
    print('matcher: ', matcher(doc))
    
    # Define variables to store label, property, and value
    label = ''
    property = ''
    value = ''
    
    # Iterate over the tokens in the input text
    for token in doc:
        # Check if the token is a label
        if len(matcher(doc)) > 0 and matcher(doc)[0][1] == token.i:
            label = token.text
            
        # Check if the token is a property
        if token.pos_ == 'NOUN' and token.i < len(doc)-2:
            if doc[token.i+1].pos_ == 'ADP' and doc[token.i+2].pos_ == 'PROPN':
                property = token.text + ' ' + doc[token.i+2].text
                
        # Check if the token is a value
        if token.pos_ == 'PROPN' or token.pos_ == 'NUM' or token.pos_ == 'ADJ':
            value += token.text + ' '
    
    # Check if a query template can be selected based on the extracted variables
    if label != '' and property != '' and value != '':
        cypher_query = pre_loaded_queries['match_nodes_by_label_and_property'].format(label=label, property=property.lower().replace(' ', '_'), value=value.strip())
    elif label != '' and property != '':
        cypher_query = pre_loaded_queries['match_nodes_by_label'].format(label=label)
    elif property != '' and value != '':
        cypher_query = pre_loaded_queries['match_nodes_by_property'].format(property=property.lower().replace(' ', '_'), value=value.strip())
    elif label == '' and property == '' and value == '':
        cypher_query = pre_loaded_queries['match_all_nodes']
    
    # Return the generated query
    return cypher_query

# Test the function with an example input
input_text = "Show me all the employees who are managers"
cypher_query = generate_cypher_query(input_text)
print(cypher_query)

doc:  Show me all the employees who are managers
matcher:  []
MATCH (n) RETURN n


In [41]:
query_templates = {
    "get_all_nodes": "MATCH (n) RETURN n",
    "get_nodes_with_label": "MATCH (n:{label}) RETURN n",
    "get_nodes_with_property": "MATCH (n:{label}) WHERE n.{property}='{value}' RETURN n",
    # add more query templates as needed
}

# define the function to generate Cypher queries from natural language input
def generate_cypher_query(natural_language_input):
    # use SpaCy to extract the relevant entities and relationships from the input
    doc = nlp(natural_language_input)
    print('doc:', doc)
    label = None
    property = None
    value = None
    # extract the label, property, and value from the input
    for ent in doc.ents:
        if ent.label_ == "LABEL":
            label = ent.text
        elif ent.label_ == "PROPERTY":
            property = ent.text
        elif ent.label_ == "VALUE":
            value = ent.text

    # choose the appropriate query template based on the extracted entities
    if label is None and property is None:
        query = query_templates["get_all_nodes"]
    elif label is not None and property is None:
        query = query_templates["get_nodes_with_label"].format(label=label)
    elif label is not None and property is not None and value is not None:
        query = query_templates["get_nodes_with_property"].format(label=label, property=property, value=value)
    else:
        # raise an error if the input is invalid
        raise ValueError("Invalid input")

    return query

# define the function to execute the Cypher query on the graph database
# def execute_cypher_query(query):
#     # connect to the graph database
#     graph = Graph("bolt://localhost:7687", auth=("username", "password"))
#     # execute the query and return the results
#     result = graph.run(query)
#     return result

# test the functions with some sample inputs
input_1 = "Get all nodes"
input_2 = "Get all nodes with label Person"
input_3 = "Get all nodes with property name='John'"

query_1 = generate_cypher_query(input_1)
query_2 = generate_cypher_query(input_2)
query_3 = generate_cypher_query(input_3)

# result_1 = execute_cypher_query(query_1)
# result_2 = execute_cypher_query(query_2)
# result_3 = execute_cypher_query(query_3)

# print(result_1.data())
# print(result_2.data())
# print(result_3.data())
print(query_1)
print(query_2)
print(query_3)

doc: Get all nodes
doc: Get all nodes with label Person
doc: Get all nodes with property name='John'
MATCH (n) RETURN n
MATCH (n) RETURN n
MATCH (n) RETURN n
